### 🔧 To-Do List

1. **Petersen Minor Graph Family**
   - [ ] Add all other graphs in the Petersen minor family.
   - [ ] Implement via [SageMath](https://www.sagemath.org/) if possible; otherwise, enter manually.

2. **Intrinsically Knotted (IK) Graph Example**
   - [ ] Find an IK example that contains $ K_6 $ or $ K_7 $.
   - 📌 *Note:* Intrinsic knotting is a minor-closed property (Conway–Foisy theorem).

3. **Intrinsically $ n $-Linked Example (for $ n \geq 3 $)**
   - [ ] Search for or construct a graph that demonstrates intrinsic $ n $-linkedness.

4. **Library of `zw_to_c_hopf`-Like Examples**
   - [ ] Collect and organize all known examples (e.g., $ (p,q) $-torus knots).
   - [ ] Store them in a `.py` file for easy import and use.
   
5. **Algorithmic Cleanliness Improvements**
   - [ ] Refactor the code to avoid repeated arguments: `thickness=0.2` and `epsilon=0.001` currently appear in both `.knot_surface_points` and `.skeleton_graph` functions.
   - [ ] Instead, pass these parameters once through the `NodalKnot` class to improve modularity and reduce redundancy.
   
6. **Surface Point Detection Improvements (Sparsity Issue)**
   - [ ] Improve surface point detection for complex knotting functions. Currently, the detected points can appear sparse, leading to incomplete or inaccurate surface representation.
   
7. **Avoid Redundant Computations in `NodalKnot.py` + Generally improve organization**
   - [ ] Investigate potential redundancy in the code—e.g., the `knot_skeleton()` function appears to recompute data that is later required by `skeleton_graph()` but doesn’t save it for reuse. (also knot_skeleton_points() uses thickness again as well)
   - [ ] If similar patterns exist elsewhere, consider caching intermediate results as class attributes to avoid unnecessary recomputation.
   - 💡 *Note:* I haven’t yet reviewed this in detail, but it’s worth checking for efficiency improvements.
   - [ ] Maybe organization can be improved in general as well(but i am not sure how yet)

8. **Check the latest yamada calculation**
   - [ ] I have checked it from multiple aspects(checking well-known examples+doing the calculation by hand for one complex knotted graph) but please you check it as well to ensure everything is correct
   - [ ] Although the Yamada calculation is largely automated, we still need to manually specify the angle to obtain the correct PD code.  
      I attempted to address this with the `find_best_view()` function, but it only works reliably in some cases.  
      While this level of functionality is sufficient for our physics project, it may be worth improving for broader dataset applications.

9. **Check whether the idea on "Thickening in Non-Hermitian Nodal Knots" can be realized in our code-with a similar figure to fig:TrefoilThickening**

10. **Find examples for the following 2 sections:**
   - [ ] Codim-1 Exceptional surfaces in 3D BZ for higher dimensional knots
   - [ ] Codim-N Exceptional surfaces in $M\>3$ BZ for higher dimensional knots

11. **Toposurface states:**
- [ ] What should we display is still a question

12. **Improve figure colorings according to prof.lee's latest suggestion**
13. **Write the draft for Skeletonization section in the paper**

# Exceptional Knots/Surfaces/Knotted Graphs

In [ ]:
import math
import numpy as np
import networkx as nx
import plotly.graph_objects as go
import matplotlib.pyplot as plt 
import matplotlib.gridspec as gridspec
from matplotlib.patches import Rectangle
import matplotlib.patches as patches
import os

try:
    import knotted_graph
except:
    ! pip install -e .
from knotted_graph import NodalKnot
from knotted_graph.yamada import optimized_yamada
from knotted_graph.pd_codes import find_best_view, PlanarDiagram_Codes
from knotted_graph.vis import draw_petersen_embedding


def k_to_zw(kx, ky, kz):
    """ F: 3D Brillouin zone -> C^2 """

    z_real = np.cos(2*kz) + 0.5
    z_imag = np.cos(kx) + np.cos(ky) + np.cos(kz) - 2.0
    z = z_real + 1j*z_imag
    
    w_real = np.sin(kx)
    w_imag = np.sin(ky)
    w = w_real + 1j*w_imag

    return z, w
def zw_to_c_hopf(z, w):
    """ f: C^2 -> C (Hopf Link) """
    return np.power(z, 2) - np.power(w, 2)

def zw_to_c_trefoil(z, w):
    """ f: C^2 -> C (Trefoil Knot) """
    return np.power(z, 2) - np.power(w, 3)

def zw_to_c_3link(z, w):
    """ f: C^2 -> C (Figure-8 Knot) """
    return np.power(z, 3) - np.power(w, 2)*z

hopf = NodalKnot(k_to_zw, zw_to_c_hopf)
trefoil = NodalKnot(k_to_zw, zw_to_c_trefoil)
threelink = NodalKnot(k_to_zw, zw_to_c_3link)

trefoil_surface = trefoil.knot_surface_points(thickness=0.2, epsilon=0.001)
trefoil_surf_fig = trefoil.plot_3D(trefoil_surface)
trefoil_surf_fig.show()
trefoil_graph = trefoil.skeleton_graph(clean=True, thickness=0.2)
trefoil_graph_fig = threelink.plot_graph(trefoil_graph)
trefoil_graph_fig.show()

In [ ]:
# 3-link Example
threelink_surface = threelink.knot_surface_points(thickness=0.05, epsilon=0.001)
threelink_surf_fig = threelink.plot_3D(threelink_surface)
threelink_surf_fig.show()

threelink_graph = threelink.skeleton_graph(clean=True, thickness=0.05)
threelink_graph_fig = threelink.plot_graph(threelink_graph)
threelink_graph_fig.show()

In [ ]:
# Arbitrary Example
def arbitrary_knotting_func(z, w):
    return (z**2+w**2)+(z**3-w**4)
k=np.pi
X = NodalKnot(k_to_zw, arbitrary_knotting_func,
              kx_min=-k, kx_max=k,
              ky_min=-k, ky_max=k,
              kz_min=0, kz_max=k,pts_per_dim=400)

X_surface = X.knot_surface_points(thickness=0.6, epsilon=0.001)
X_surf_fig = X.plot_3D(X_surface)
X_surf_fig.show()

X_points = X.knot_skeleton_points() 
X_graph = X.skeleton_graph(clean=True, thickness=0.6)
X_graph_fig = X.plot_graph(X_graph)
X_graph_fig.show()

#### Intrinsically Linked - Petersen Graph Minor Example

In [ ]:
def arbitrary_func(z, w):
    """ f: C^2 -> C (Figure-8 Knot) """   
    return  z*(z**2-w**4+w)
 
k=0.9*np.pi
X = NodalKnot(k_to_zw, arbitrary_func,
              kx_min=-k, kx_max=k,
              ky_min=-k, ky_max=k,
              kz_min=0, kz_max=k,pts_per_dim=600)
X_surface = X.knot_surface_points(thickness=0.2, epsilon=0.001)


X_points = X.knot_skeleton_points()
X_fig = trefoil.plot_3D(X_points)
X_fig.show()


X_surf_fig = X.plot_3D(X_surface)
X_surf_fig.show()


X_graph = X.skeleton_graph(clean=True, thickness=0.2)
X_graph_fig = X.plot_graph(X_graph)
X_graph_fig.show()


# Generate positions using the shell layout.
pos = nx.shell_layout(X_graph)
plt.figure(figsize=(4, 4))
nx.draw(X_graph, pos, with_labels=True,
        node_color='white', edgecolors='black',  # white fill, black border for nodes
        edge_color='black', node_size=500) 
plt.show()


X.print_graph_properties(X_graph)

petersen_graph= nx.petersen_graph() ### Try to add whole petersen family of graphs to create a function Is_Intrinsically_Linked
Embedding=X.check_minor(host_graph=X_graph,minor_graph=petersen_graph)

 
fig, ax = draw_petersen_embedding(petersen_graph, Embedding)
plt.show() 

#### Ways Of Introducing Non-Hermitian Thickening 

In [ ]:

# pick a single thickness
th = -0.5

# compute the three surfaces
surf1 = threelink.knot_surface_points(thickness=[th, 0, 0], epsilon=0.01)
surf2 = threelink.knot_surface_points(thickness=[0, th, 0], epsilon=0.01)
surf3 = threelink.knot_surface_points(thickness=[0, 0, th], epsilon=0.01)

# set up figure with 3 side-by-side 3D plots
fig = plt.figure(figsize=(15, 5))
for i, surf in enumerate((surf1, surf2, surf3), start=1):
    ax = fig.add_subplot(1, 3, i, projection='3d')
    ax.scatter(surf[:,0], surf[:,1], surf[:,2],
               c='blue', s=10, alpha=0.7, rasterized=True)
    ax.set_xticks([]); ax.set_yticks([]); ax.set_zticks([])
    ax.set_title([r'$c$', r'$c\,\mathrm{Re}[f(\mathbf{k})]$', r'$c\,\mathrm{Im}[f(\mathbf{k})]$'][i-1],
                 fontsize=16)
    ax.view_init(elev=30, azim=45)

plt.tight_layout()
plt.show()

# Yamada Polynomial

In [ ]:
def arbitrary_func(z, w):
    """ f: C^2 -> C (Figure-8 Knot) """   
    return  z*(z**2-w**4+w)
 
k=0.9*np.pi
X = NodalKnot(k_to_zw, arbitrary_func,
              kx_min=-k, kx_max=k,
              ky_min=-k, ky_max=k,
              kz_min=0, kz_max=k,pts_per_dim=600)
graph = X.skeleton_graph(clean=True, thickness=0.6)
graphfig = threelink.plot_graph(graph)
graphfig.show()

##### Note: The function below(find_best_vies) works in some cases but may fail in others(in such cases manual angle should be given).


In [ ]:
best_view= find_best_view(graph,
    max_pts=20,
    init_view=(0, 30),
    T0=10000,
    Tmin=0.5,
    alpha=0.8,
    steps=50,
    tol=3.0,
    cross_penal_factor=4,
    cross_penal_dist=5.0,
    cross_dist_penal_factor=155,
    node_penal_dist=5.0,
    node_penal_factor=15,
)### Start from 5 diff point and optimize in parallel

In [ ]:
# Generate planar diagram parts
V_parts, X_parts,meet = PlanarDiagram_Codes(graph, view=(0,40),crossing_tol=5) # can enter view=best_view
# Combine into PD code
pd_code = ";".join(V_parts + X_parts)

In [ ]:
optimized_yamada(pd_code)